# .resample()

Теперь посмотрим, какие возможности pandas предоставляет для работы с временными рядами.

Один из наиболее часто используемых и удобных методов — `.resample()`, позволяющий преобразовать данные и применить к ним другой метод (`sum()`, `size()` и пр.). Таким образом, можно рассчитать показатели, например, за весь день, неделю, месяц и т.п. С полным списком возможных значений можно ознакомиться здесь:

https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects

Например, посчитать сумму показателя по дням, имея данные по часам, можно следующим образом:

`data.resample(rule='D').sum()`

где `rule` — параметр, отвечающий за то, по какому периоду нужно агрегировать данные. В данном случае параметр он равен `'D'` (Day). 

Для получения данных за каждые 6 часов используем число и обозначение `H`:

`data.resample(rule='6H').sum()`

В качестве индексов датафрейма обязательно нужно использовать колонку формата `DateTime`, отсортированную в правильном порядке. Поэтому предварительно всегда следует проверить правильность типа данных и, если требуется, привести его к правильному с помощью `pd.to_datetime()`.

In [1]:
import pandas as pd
import numpy  as np

В следующих задачах мы будем работать сэмплом данных об аренде велосипедов в Чикаго:

- **trip_id** — id поездки;
- **start_time** — Дата и время начала поездки
- **end_time** — Дата и время конца поездки
- **bikeid** — id велосипеда
- **tripduration** — продолжительность поездки в минутах
- **from_station_id** — id станции начала поездки
- **from_station_name** — название пункта отправления
- **to_station_id** — id пункта прибытия
- **to_station_name** — название пункта прибытия
- **usertype** — тип пользователя
- **gender** — пол (если subscriber)
- **birthyear** — год рождения (если subscriber)

In [2]:
# bikes_Q1 = pd.read_csv('https://stepik.org/media/attachments/lesson/367415/bikes_q1_sample.csv')
bikes_Q1 = pd.read_csv('bikes_q1_sample.csv')

In [3]:
bikes_Q1.head()

,trip_id,start_time,end_time,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthyear
0,17617135,2018-01-22 20:04:31,2018-01-22 20:11:53,1131,442.0,471,Francisco Ave & Foster Ave,468,Budlong Woods Library,Subscriber,Female,1949.0
1,17897619,2018-03-16 19:47:59,2018-03-16 20:04:00,6146,961.0,296,Broadway & Belmont Ave,253,Winthrop Ave & Lawrence Ave,Subscriber,Male,1988.0
2,17881307,2018-03-14 18:49:20,2018-03-14 18:54:38,3847,318.0,260,Kedzie Ave & Milwaukee Ave,503,Drake Ave & Fullerton Ave,Subscriber,Male,1987.0
3,17881130,2018-03-14 18:33:48,2018-03-14 19:07:40,1483,"2,032.0",199,Wabash Ave & Grand Ave,199,Wabash Ave & Grand Ave,Subscriber,Male,1990.0
4,17686289,2018-02-05 17:39:14,2018-02-05 17:46:13,6391,419.0,596,Benson Ave & Church St,605,University Library (NU),Subscriber,Male,1992.0


In [4]:
# bikes_Q1.shape

In [5]:
# bikes_Q1.dtypes

In [6]:
# bikes_Q1.info()

Для начала, возьмем данные только за Q1, они уже сохранены в переменную `bikes_Q1`. Перед тем как сделать `.resample()`, нужно немного подготовить данные. Поместите колонку `start_time` в качестве индексов и сохраните изменения в исходный датасет. Предварительно проверьте тип переменной, и приведите её к правильному, если необходимо.

как привести к DateTime?
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html 

In [7]:
# bikes_Q1['start_time'] = pd.to_datetime(bikes_Q1['start_time'])

In [8]:
bikes_Q1[['start_time', 'end_time']] = bikes_Q1[['start_time', 'end_time']].apply(pd.to_datetime)
bikes_Q1 = bikes_Q1.sort_values('start_time').set_index('start_time')
bikes_Q1.head()

,trip_id,end_time,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthyear
start_time,,,,,,,,,,,
2018-01-01 00:44:46,17536704,2018-01-01 01:33:10,4599,"2,904.0",98,LaSalle St & Washington St,509,Troy St & North Ave,Subscriber,Male,1989.0
2018-01-01 00:57:26,17536708,2018-01-01 01:02:40,1169,314.0,164,Franklin St & Lake St,174,Canal St & Madison St,Subscriber,Male,1998.0
2018-01-01 01:07:54,17536711,2018-01-06 10:04:02,4783,"464,168.0",99,Lake Shore Dr & Ohio St,99,Lake Shore Dr & Ohio St,Customer,NaN,NaN
2018-01-01 01:15:59,17536715,2018-01-01 01:21:45,6330,346.0,85,Michigan Ave & Oak St,106,State St & Pearson St,Subscriber,Female,1993.0
2018-01-01 01:25:15,17536718,2018-01-01 01:36:32,6099,677.0,462,Winchester (Ravenswood) Ave & Balmoral Ave,238,Wolcott (Ravenswood) Ave & Montrose Ave (*),Subscriber,Male,1979.0


В данных имеется как дата аренды, так и её точное время начала и окончания с точностью до секунд.
Примените метод `pd.resample()` и агрегируйте данные по дням.

**В качестве ответа укажите максимальное число аренд за день.**

In [9]:
bikes_Q1.resample(rule = 'D').size().max()

4196

Посмотрим на распределение количества аренд для разных групп пользователей (`usertype`) — `customers` и `subscribers` в данных за апрель. Данные за нужный период можно подгрузить отсюда.
https://stepik.org/media/attachments/lesson/367415/bikes_april.csv

Сделайте ресемпл по дням для каждой группы и в качестве ответа укажите число аренд за 18 апреля, сделанных пользователями типа Subscriber.

Может пригодиться:

функция для транспонирования `.T`
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.transpose.html

In [10]:
# bikes_april = pd.read_csv('https://stepik.org/media/attachments/lesson/367415/bikes_april.csv')
bikes_april = pd.read_csv('bikes_april.csv')
bikes_april[['start_time', 'end_time']] = bikes_april[['start_time', 'end_time']].apply(pd.to_datetime)
bikes_april = bikes_april.sort_values('start_time').set_index('start_time')
bikes_april.head()

,trip_id,end_time,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthyear
start_time,,,,,,,,,,,
2018-04-01 00:10:23,18000531,2018-04-01 00:22:12,5065,709.0,228,Damen Ave & Melrose Ave,219,Damen Ave & Cortland St,Subscriber,Male,1983.0
2018-04-01 00:15:49,18000533,2018-04-01 00:19:47,4570,238.0,128,Damen Ave & Chicago Ave,130,Damen Ave & Division St,Subscriber,Male,1978.0
2018-04-01 00:17:00,18000534,2018-04-01 00:22:53,1323,353.0,130,Damen Ave & Division St,69,Damen Ave & Pierce Ave,Subscriber,Male,1991.0
2018-04-01 00:20:00,18000536,2018-04-01 00:26:22,2602,382.0,121,Blackstone Ave & Hyde Park Blvd,351,Cottage Grove Ave & 51st St,Subscriber,Female,1992.0
2018-04-01 00:23:19,18000538,2018-04-01 00:35:01,4213,702.0,31,Franklin St & Chicago Ave,180,Ritchie Ct & Banks St,Subscriber,Male,1985.0


In [11]:
# Не самое эстетичное решение
# bikes_april.groupby('usertype').resample(rule = 'D').trip_id.count()

In [12]:
# Симпатичная выдача
data = (bikes_april
            .groupby('usertype')
            .resample(rule = 'D')
            .size()
            .T
       )
data.head()

usertype,Customer,Subscriber
start_time,,
2018-04-01,239,825
2018-04-02,166,2841
2018-04-03,31,1873
2018-04-04,82,2253
2018-04-05,90,2502


In [13]:
data.loc['2018-04-18', 'Subscriber']

2196

## ⭐️ Посмотрим на данные за период с апреля по декабрь.

Объедините сэмплы данных за нужные месяцы в один общий датасет `bikes`. Сделайте преобразование по дням для каждой группы пользователей (`usertype`), затем выберите дни, в которые число аренд, сделанных `customers`, было больше, чем у `subscribers`.

Данные за:

- Q2:
    - (1) апрель `bikes_q2_sample_apr.csv` https://stepik.org/media/attachments/lesson/367415/bikes_q2_sample_apr.csv
    - (2) май `bikes_q2_sample_may.csv` https://stepik.org/media/attachments/lesson/367415/bikes_q2_sample_may.csv
    - (3) июнь `bikes_q2_sample_jun.csv` https://stepik.org/media/attachments/lesson/367415/bikes_q2_sample_jun.csv
- Q3:
    - (4) июль `bikes_q3_sample_july.csv` https://stepik.org/media/attachments/lesson/367415/bikes_q3_sample_july.csv
    - (5) август `bikes_q3_sample_aug.csv` https://stepik.org/media/attachments/lesson/367415/bikes_q3_sample_aug.csv
    - (6) сентябрь `bikes_q3_sample_sep.csv` https://stepik.org/media/attachments/lesson/367415/bikes_q3_sample_sep.csv
- Q4:
    - (7) октябрь `bikes_q4_sample_oct.csv` https://stepik.org/media/attachments/lesson/367415/bikes_q4_sample_oct.csv
    - (8) ноябрь `bikes_q4_sample_nov.csv` https://stepik.org/media/attachments/lesson/367415/bikes_q4_sample_nov.csv
    - (9) декабрь `bikes_q4_sample_dec.csv` https://stepik.org/media/attachments/lesson/367415/bikes_q4_sample_dec.csv

***Note:*** объединение данных может занять немного времени, don't worry

In [14]:
bikes = pd.concat([
                   pd.read_csv("bikes_q2_sample_apr.csv"), 
                   pd.read_csv("bikes_q2_sample_may.csv"), 
                   pd.read_csv("bikes_q2_sample_jun.csv"), 
                   pd.read_csv("bikes_q3_sample_july.csv"),
                   pd.read_csv("bikes_q3_sample_aug.csv"), 
                   pd.read_csv("bikes_q3_sample_sep.csv"), 
                   pd.read_csv("bikes_q4_sample_oct.csv"), 
                   pd.read_csv("bikes_q4_sample_nov.csv"),
                   pd.read_csv("bikes_q4_sample_dec.csv")
                    ])
bikes.head()

,trip_id,start_time,end_time,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthyear
0,18000534,2018-04-01 00:17:00,2018-04-01 00:22:53,1323,353.0,130,Damen Ave & Division St,69,Damen Ave & Pierce Ave,Subscriber,Male,1991.0
1,18000536,2018-04-01 00:20:00,2018-04-01 00:26:22,2602,382.0,121,Blackstone Ave & Hyde Park Blvd,351,Cottage Grove Ave & 51st St,Subscriber,Female,1992.0
2,18000538,2018-04-01 00:23:19,2018-04-01 00:35:01,4213,702.0,31,Franklin St & Chicago Ave,180,Ritchie Ct & Banks St,Subscriber,Male,1985.0
3,18000540,2018-04-01 00:24:46,2018-04-01 00:44:23,6401,"1,177.0",596,Benson Ave & Church St,517,Clark St & Jarvis Ave,Subscriber,Male,1974.0
4,18000541,2018-04-01 00:26:04,2018-04-01 00:31:05,6333,301.0,145,Mies van der Rohe Way & Chestnut St,24,Fairbanks Ct & Grand Ave,Subscriber,Male,1984.0


In [15]:
# bikes.shape

In [16]:
bikes[['start_time', 'end_time']] = bikes[['start_time', 'end_time']].apply(pd.to_datetime)
bikes = bikes.sort_values('start_time').set_index('start_time')

In [17]:
data = (bikes
            .groupby('usertype')
            .resample(rule = 'D')
            .size()
            .T
       )
data.query("Customer > Subscriber")

usertype,Customer,Subscriber
start_time,,
2018-05-27,3263,2449
2018-09-02,2752,2183


## Летний период
**Отлично!** Следующий шаг – более подробно посмотреть на летний период.

Еще один плюс использования дат в качестве индексов – возможность выбрать данные за интересующий нас промежуток времени.

В переменную `bikes_summer` сохраните наблюдения с 1 июня по 31 августа. Затем запишите в `top_destination` наиболее популярный пункт назначения (его название).

Агрегируйте данные по дням и определите, в какой день в полученный пункт (top_destination) было совершено меньше всего поездок. 

Дату сохраните в `bad_day`, отформатировав `timestamp` с помощью `.strftime('%Y-%m-%d')`.

Могут пригодиться:
- `loc`
- `strftime`
- `idxmin`, `idxmax`
- `size`
- `query`

*Полный датафрейм уже сохранен как `bikes`, еще раз объединять данные в этом степе не нужно :)
В качестве индексов используется `start_time` в нужном формате.*

In [18]:
# В переменную bikes_summer сохраните наблюдения с 1 июня по 31 августа.
bikes_summer = bikes.loc['2018-06-01':'2018-08-31']
bikes_summer.head()

,trip_id,end_time,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthyear
start_time,,,,,,,,,,,
2018-06-01 00:04:40,18709077,2018-06-01 00:06:47,3155,127.0,128,Damen Ave & Chicago Ave,214,Damen Ave & Grand Ave,Subscriber,Female,1978.0
2018-06-01 00:06:08,18709080,2018-06-01 00:24:18,2807,"1,090.0",258,Logan Blvd & Elston Ave,69,Damen Ave & Pierce Ave,Customer,NaN,NaN
2018-06-01 00:08:01,18709086,2018-06-01 00:32:55,2737,"1,494.0",337,Clark St & Chicago Ave,225,Halsted St & Dickens Ave,Customer,Male,1988.0
2018-06-01 00:09:02,18709091,2018-06-01 00:19:21,6089,619.0,210,Ashland Ave & Division St,56,Desplaines St & Kinzie St,Subscriber,Male,1987.0
2018-06-01 00:09:28,18709092,2018-06-01 00:14:44,2352,316.0,240,Sheridan Rd & Irving Park Rd,303,Broadway & Cornelia Ave,Subscriber,Male,1997.0


In [19]:
# в top_destination наиболее популярный пункт назначения (его название).
top_destination = (bikes_summer
                       .to_station_name
                       .value_counts()
                       .idxmax()
                  )
top_destination

'Streeter Dr & Grand Ave'

In [20]:
# в какой день в полученный пункт (top_destination) было совершено меньше всего поездок
bad_day = (bikes_summer
               .query("to_station_name == @top_destination")
               .resample(rule = 'D')
               .size()
               .idxmin()
               .strftime('%Y-%m-%d')
          )
bad_day

'2018-06-21'

## Куда больше всего ездят на выходных?
Туда же, куда и в будние дни, или в другие пункты назначения?

Используя данные за период с 1 июня по 31 августа, выберите верные утверждения.

***Hint***: Как получить день недели?
https://stackoverflow.com/questions/29096381/num-day-to-name-day-with-pandas

In [21]:
bikes_summer['weekday'] = bikes_summer.index.weekday < 5
bikes_summer['day_name'] = bikes_summer.index.day_name()
bikes_summer.head()

C:\Users\A\AppData\Local\Temp/ipykernel_1012/293958544.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bikes_summer['weekday'] = bikes_summer.index.weekday < 5
C:\Users\A\AppData\Local\Temp/ipykernel_1012/293958544.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bikes_summer['day_name'] = bikes_summer.index.day_name()


,trip_id,end_time,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthyear,weekday,day_name
start_time,,,,,,,,,,,,,
2018-06-01 00:04:40,18709077,2018-06-01 00:06:47,3155,127.0,128,Damen Ave & Chicago Ave,214,Damen Ave & Grand Ave,Subscriber,Female,1978.0,True,Friday
2018-06-01 00:06:08,18709080,2018-06-01 00:24:18,2807,"1,090.0",258,Logan Blvd & Elston Ave,69,Damen Ave & Pierce Ave,Customer,NaN,NaN,True,Friday
2018-06-01 00:08:01,18709086,2018-06-01 00:32:55,2737,"1,494.0",337,Clark St & Chicago Ave,225,Halsted St & Dickens Ave,Customer,Male,1988.0,True,Friday
2018-06-01 00:09:02,18709091,2018-06-01 00:19:21,6089,619.0,210,Ashland Ave & Division St,56,Desplaines St & Kinzie St,Subscriber,Male,1987.0,True,Friday
2018-06-01 00:09:28,18709092,2018-06-01 00:14:44,2352,316.0,240,Sheridan Rd & Irving Park Rd,303,Broadway & Cornelia Ave,Subscriber,Male,1997.0,True,Friday


### Выберите верные утверждения:
- **Наиболее популярный пункт назначения в субботу – Streeter Dr & Grand Ave**
- **Второй по числу поездок в субботу – Lake Shore Dr & North Blvd**
- Второй по числу поездок в субботу – Canal St & Adams St
- В будние дни больше всего поездок совершено до Canal St & Adams St
- **В будние дни больше всего поездок совершается до Streeter Dr & Grand Ave**

In [22]:
(
    bikes_summer
        .groupby(['weekday', 'day_name'])
        .to_station_name
        .value_counts()
        .nlargest(10)
)

weekday  day_name   to_station_name           
False    Saturday   Streeter Dr & Grand Ave       3461
         Sunday     Streeter Dr & Grand Ave       2565
True     Friday     Streeter Dr & Grand Ave       1726
False    Saturday   Lake Shore Dr & North Blvd    1690
True     Wednesday  Streeter Dr & Grand Ave       1669
         Monday     Streeter Dr & Grand Ave       1503
         Thursday   Streeter Dr & Grand Ave       1482
False    Saturday   Theater on the Lake           1470
         Sunday     Lake Shore Dr & North Blvd    1402
                    Theater on the Lake           1388
Name: to_station_name, dtype: int64